In [18]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import time
from keras.preprocessing.image import save_img

## Download the Simpsons datset. 
### The orginal dataset can be downloaded from https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

In [ ]:
!wget https://www.dropbox.com/s/cq4jajywga9eeut/simpsons.zip?dl=0
!mv simpsons.zip\?dl\=0 simpsons.zip
!unzip simpsons.zip
!rm simpsons.zip

In [26]:
# dimensions of our images.
img_width, img_height = 224, 224 

In [27]:
train_data_dir = '/home/setuc/notebooks/data/simpsons/train/'
validation_data_dir = '/home/setuc/notebooks/data/simpsons/valid/'
nb_train_samples = 31688
nb_validation_samples = 10657
batch_size = 64

In [28]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [29]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 16694 images belonging to 47 classes.
Found 4239 images belonging to 47 classes.


## Create the ResNet50 Model for transfer learning

In [30]:
inception_base = applications.ResNet50(weights='imagenet', include_top=False)

/anaconda/envs/py35/lib/python3.5/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [31]:
x = inception_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(47, activation='softmax')(x)
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

In [32]:
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
inception_transfer.fit_generator(
    train_generator,
    steps_per_epoch=16694 // 64,
    epochs=5, shuffle = True, verbose = 1, 
    max_queue_size=10,
    validation_data=validation_generator,
	validation_steps=4239 // 64)

Epoch 1/5
159/517 [========>.....................] - ETA: 12:55 - loss: 3.4418 - acc: 0.1171